In [3]:
import netCDF4 as nc
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import os
import math
#import imageio
from scipy.ndimage.interpolation import map_coordinates
from pprint import pprint
import glob
from scipy import interpolate
from scipy.interpolate import RegularGridInterpolator
from copy import copy
from copy import deepcopy
#from mpl_toolkits.axes_grid1 import make
#_axes_locatable
%matplotlib inline
from scipy import stats
import multiprocessing as mp
import matplotlib.patches as patches

In [4]:
_wbgyr_cdict = {'red': ((0.0,  255./255, 255./255),
                        (0.125, 173./255, 173./255),
                        (0.25,  95./255,  95./255),
                        (0.375, 73./255,  73./255),
                        (0.5,  165./255, 164./255),
                        (0.625, 248./255, 248./255),
                        (0.75, 236./255, 236./255),
                        (0.875, 200./255, 200./255),
                        (1.0,  146./255, 146./255)),
                'green': ((0.0,  255./255, 255./255),
                          (0.125, 224./255, 224./255),
                          (0.25, 163./255, 163./255),
                          (0.375, 166./255, 166./255),
                          (0.5,  207./255, 207./255),
                          (0.625, 184./255, 184./255),
                          (0.75,  86./255,  86./255),
                          (0.875, 29./255,  29./255),
                          (1.0,   21./255,  21./255)),
                'blue': ((0.0,  255./255, 255./255),
                         (0.125, 248./255, 248./255),
                         (0.25, 214./255, 214./255),
                         (0.375, 120./255, 120./255),
                         (0.5,   81./255,  81./255),
                         (0.625, 73./255,  73./255),
                         (0.75,  41./255,  41./255),
                         (0.875, 38./255,  38./255),
                         (1.0,   25./255,  25./255))}
# And reverse
_wbgyr_cdict_r = deepcopy(_wbgyr_cdict)
for i in _wbgyr_cdict_r.keys():
    _wbgyr_cdict_r[i] = [(1-j[0], j[1], j[2]) for j in _wbgyr_cdict_r[i]]
    _wbgyr_cdict_r[i].reverse()

cmap = LinearSegmentedColormap('WBGYR', _wbgyr_cdict)
cmap.set_bad('#D2D2D2')
plt.register_cmap(cmap=cmap)

cmap_r = LinearSegmentedColormap('WBGYR', _wbgyr_cdict_r)
cmap_r.set_bad('#D2D2D2')
plt.register_cmap(cmap=cmap_r)

In [5]:
import warnings
print('is it doing?')

yedges_net = np.arange(-650,650,1)
yedges_lw = np.arange(-500,0,1)
yedges_sw = np.arange(-100,900,1)
yedges_solar = np.arange(0,1400,5)
xedges = np.arange(0,500,1)

cereshour = 8760


k=0

year = 2008

tsc = xr.open_mfdataset('/net/seldon/disk2/Users/gah20/TSCISCCP/TSC_convolve/'+str(year)+'/TSC*.nc',combine='nested',concat_dim='time')['TSC'][:,2:62]
landocean = xr.open_mfdataset('/net/seldon/disk2/Users/gah20/TSCISCCP/TSC_convolve/'+str(year)+'/TSC*.nc',combine='nested',concat_dim='time')['ConvOrigin'][:,2:62]
aodtsc = xr.open_mfdataset('/net/seldon/disk2/Users/gah20/TSCISCCP/TSC_convolve/'+str(year)+'/TSC*.nc',combine='nested',concat_dim='time')['ConvAOD'][:,2:62]
detinsit = xr.open_mfdataset('/net/seldon/disk2/Users/gah20/TSCISCCP/TSC_testdelete/'+str(year)+'/TSC*.nc',combine='nested',concat_dim='time')['ConvCir'][:,2:62]

isccp = xr.open_mfdataset('/net/seldon/disk2/Users/gah20/ISCCP/access/isccp-basic/hgg/'+str(year)+'*/200*.nc',combine='nested',concat_dim='time')
taupc = isccp['n_pctaudist']
levtau = isccp['levtau']
levpc = isccp['levpc']
tauraw = isccp['tau']
pcraw = isccp['pc']
isccptime = isccp['time']

ds_w = xr.open_mfdataset('/net/seldon/disk1/Data/CERES/SYN1deg-Hour-Ed4/'+str(year)+'/CERES_SYN1deg-1H_Terra-Aqua-MODIS_Ed4.1_Total_*.nc',combine = 'by_coords', concat_dim="time")
swall = ds_w['toa_sw_all_1h'][:]
swclr = ds_w['toa_sw_clr_1h'][:]
lwall = ds_w['toa_lw_all_1h'][:]
lwclr = ds_w['toa_lw_clr_1h'][:]
netall = ds_w['toa_net_all_1h'][:]
netclr = ds_w['toa_net_clr_1h'][:]
solar = ds_w['toa_solar_all_1h'][:]
timeceres = ds_w['time'][:]
cf = ds_w['cldarea_total_1h'][:]

offset = np.argmin(np.abs(isccp['time'][:].values - ds_w['time'][0].values))

meanlw = np.nanmean(lwclr[:],axis=0)

lowcloudraw = np.nansum(np.nansum(isccp['n_pctaudist'][offset:,:,3:,60:120],axis=1),axis=1)
lowcloud_day = np.where(lowcloudraw<0,np.nan,lowcloudraw)
lowcloud_night = np.where(np.nansum(isccp['n_pcdist'][offset:,3:,60:120],axis=1)>100,np.nan,np.nansum(isccp['n_pcdist'][offset:,3:,60:120],axis=1))
lowcloud = np.where(np.isnan(lowcloud_day),lowcloud_night,lowcloud_day)


timeceres[0::3][0:len(isccp['time'][offset:])]


swcldy  = (swall[0::3][0:len(isccp['time'][offset:])] - swclr[0::3][0:len(isccp['time'][offset:])])
netcldy = (netall[0::3][0:len(isccp['time'][offset:])] - netclr[0::3][0:len(isccp['time'][offset:])])

lwclr_f = np.where(np.isnan(lwclr[0::3][0:len(isccp['time'][offset:])]),meanlw,lwclr[0::3][0:len(isccp['time'][offset:])])
lwcldy = (lwall[0::3][0:len(isccp['time'][offset:])] - lwclr_f) 
solar_t = solar[0::3][0:len(isccp['time'][offset:])]
swcldy_dayf = np.where((lowcloud_day<1),swcldy,np.nan)
lwcldy_dayf = np.where((lowcloud_day<1),lwcldy,np.nan)

swcldy_f_nans = np.where((lowcloud<1),swcldy,np.nan)
lwcldy_f = np.where((lowcloud<1),lwcldy,np.nan)
tsc_t = (np.asarray(tsc[offset*3::3]))

tsc_d = np.asarray(np.where(detinsit>0,tsc,np.nan)[offset*3::3])
tsc_i = np.asarray(np.where(detinsit<0,tsc,np.nan)[offset*3::3])

swcldy_dayf = np.where(solar_t>0,swcldy_dayf,0)
swcldy_f = np.where(solar_t>0,swcldy_f_nans,0)
swcldy_fg = np.where(lowcloud<1,swcldy_f,np.nan)

H_sw_o_det, xedges, yedges_sw = np.histogram2d(tsc_t.flatten(),np.asarray(swcldy_f).flatten(), bins=(xedges,yedges_sw))
H_lw_o_det, xedges, yedges_lw = np.histogram2d(tsc_t.flatten(),np.asarray(lwcldy_f).flatten(), bins=(xedges,yedges_lw))

H_sw_o_ins, xedges, yedges_sw = np.histogram2d(tsc_i.flatten(),np.asarray(swcldy_f).flatten(), bins=(xedges,yedges_sw))
H_lw_o_ins, xedges, yedges_lw = np.histogram2d(tsc_i.flatten(),np.asarray(lwcldy_f).flatten(), bins=(xedges,yedges_lw))

H_sw_o_all, xedges, yedges_sw = np.histogram2d(tsc_t.flatten(),np.asarray(swcldy_fg).flatten(), bins=(xedges,yedges_sw))
H_lw_o_all, xedges, yedges_lw = np.histogram2d(tsc_t.flatten(),np.asarray(lwcldy_f).flatten(), bins=(xedges,yedges_lw))
#H0_solar_o, xedges, yedges_solar =  np.histogram2d(tsc_t.flatten(),np.asarray(solar_t).flatten(), bins=(xedges,yedges_solar))



is it doing?


/home/gah20/.conda/envs/py3_jl/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice
